In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from torch.utils.data import DataLoader
from unet import UNet
from data_loading import XRayDataset2
from data_loading import XRayDataset
import lightning as L
import pathlib
import torch
import data_processing
import numpy as np
from lightning.pytorch.callbacks import TQDMProgressBar
from torchvision import transforms
import matplotlib.pyplot as plt
from lightning.pytorch.loggers import TensorBoardLogger
import lightning as L
import skimage as ski
import worker_seed
import loss
import os
import posixpath
import glob
import cv2
from torchvision.transforms import v2
from numpy import asarray


# Load data


In [ ]:
xray_dataset = XRayDataset2(pathlib.Path.cwd() / "output/yolo_output", data_processing.preprocess_inputs)

In [ ]:
print(xray_dataset.__len__())

# Load model

In [ ]:
checkpoint = torch.load("./checkpoints/ckp_yolo_aug_unet.ckpt", map_location=torch.device('cpu'))
loaded = UNet(loss.Loss)
loaded.load_state_dict(checkpoint["state_dict"])

In [ ]:
model = loaded.eval()

# Run inference

In [ ]:
from pathlib import Path
Path("./output/inference").mkdir(parents=True, exist_ok=True)

for i in range(len(xray_dataset)):
    sample_input = xray_dataset.__getitem__(i)
    sample_input_filename = xray_dataset.get_filename(i)
    model.eval()
    with torch.no_grad():
        predictions = model.forward(sample_input.unsqueeze(0))
    base, ext = os.path.splitext(os.path.basename(sample_input_filename))  
    ext = ext.lower()
    input = data_processing.resize_to_roughy_input_size(sample_input.squeeze()).permute(1, 2, 0)
    prediction = data_processing.postprocess(data_processing.resize_to_roughy_input_size(predictions.squeeze()))
    print('.', end='')
    cv2.imwrite(f"./output/inference/{base}_prediction{ext}", prediction)

print(".")
print("finished")